In [10]:
import helpers
import dataset

import pandas as pd
import tensorflow_decision_forests as tfdf
from sklearn.ensemble import RandomForestClassifier

truthPath = './processed_truth/'
liePath = './processed_lie/'

testRatio = 0.2

Train, Xtrain, Ytrain, Xtest, Ytest = dataset.createDatasetGeneral(truthPath, liePath, testRatio)

In [ ]:
def getTFRF():

    dataset = tfdf.keras.pd_dataframe_to_tf_dataset(Train, label="Result")

    #https://www.tensorflow.org/decision_forests/api_docs/python/tfdf/keras/RandomForestModel
    TFmodel = tfdf.keras.RandomForestModel() #add params
    TFmodel.compile()
    TFmodel.fit(dataset)

    print(TFmodel.summary())

    res = pd.DataFrame(TFmodel.predict(tfdf.keras.pd_dataframe_to_tf_dataset(Xtest)))

    for i in range(res.shape[0]):
        if res.iloc[i,0] > 0.5: 
            res.iloc[i] = 1 
        else:
            res.iloc[i] = 0

    helpers.displayConfusion(Ytest, res)

    return TFmodel

In [ ]:
def getSKRF():

    #https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
    SKRF = RandomForestClassifier(verbose=2) #add params
    SKRF.fit(Xtrain, Ytrain)

    helpers.displayConfusion(Ytest, SKRF.predict(Xtest))

    return SKRF

In [ ]:
#run the models here
